In [1]:
import numpy as np
import cvxpy as cp
from general_method.basis_generator import rand_moment, generate_basis, sel_seq

# Define the Scenario

In [2]:
num_obs = 3
len_seq = 2
out_max = 1
seq_method = "sel_sequences"
remove_last_out = True

In [3]:
sequences = sel_seq(n = num_obs,
                    r = len_seq,
                    out_max = out_max,
                    remove_last_out = remove_last_out)

In [4]:
print(len(sequences))
sequences

9


[((0, 0), (0, 0)),
 ((0, 0), (0, 1)),
 ((0, 0), (0, 2)),
 ((0, 0), (1, 0)),
 ((0, 0), (1, 1)),
 ((0, 0), (1, 2)),
 ((0, 0), (2, 0)),
 ((0, 0), (2, 1)),
 ((0, 0), (2, 2))]

# Sampling Q3 

In [5]:
dim = 3
sel_sequences = [len_seq]

In [6]:
batch_size = 100
X_basis_3d, rank = generate_basis(dim, 
                                  num_obs, 
                                  len_seq, 
                                  out_max, 
                                  batch_size, 
                                  seq_method = seq_method,
                                  sel_sequences = sel_sequences,
                                  remove_last_out=remove_last_out)

rank

46

In [7]:
X = rand_moment(dim, num_obs, len_seq, out_max, seq_method, sel_sequences ,remove_last_out=remove_last_out)
X.shape

(10, 10)

# Generate Q2,NPAk set

In [8]:
dim = 2
level = 1
batch_size = 100

batch_size

100

In [9]:
if level > 1:
    hierarchy_seq = sequences + sel_seq(n = num_obs,
                                    r = len_seq + level -1,
                                    out_max = out_max,
                                    remove_last_out = remove_last_out)
else:
    hierarchy_seq = sequences
    
print(len(hierarchy_seq))
hierarchy_seq

9


[((0, 0), (0, 0)),
 ((0, 0), (0, 1)),
 ((0, 0), (0, 2)),
 ((0, 0), (1, 0)),
 ((0, 0), (1, 1)),
 ((0, 0), (1, 2)),
 ((0, 0), (2, 0)),
 ((0, 0), (2, 1)),
 ((0, 0), (2, 2))]

In [10]:
X_basis_2d, rank = generate_basis(dim = dim, 
                                  num_obs = num_obs, 
                                  len_seq= len_seq + level - 1,
                                  out_max = out_max,
                                  batch_size = batch_size,
                                  seq_method=seq_method,
                                  sel_sequences = [len_seq, len_seq + level - 1],
                                  remove_last_out=remove_last_out)

print(X_basis_2d[0].shape)
rank

(10, 10)


35

In [11]:
#X_basis_2d = []

for i in range(500):
    X_basis_2d.append(rand_moment(dim, num_obs, len_seq, out_max, seq_method, [len_seq, len_seq + level - 1], remove_last_out=remove_last_out))

# Testing via SDP

In [12]:
eta = cp.Variable((1, 1))
alpha = cp.Variable((len(X_basis_2d), 1))

constraints = [sum([alpha[j]*X_basis_2d[j] for j in range(len(X_basis_2d))]) >> 0]
constraints += [sum([alpha[j]*X_basis_2d[j] for j in range(len(X_basis_2d))])[0,0] == 1]
for i in range(1,len(X)):
    constraints += [
        #eta*X[i,i] + (1-eta)*X_basis_2d[0][i,i] == sum([alpha[j]*X_basis_2d[j][i,i] for j in range(len(X_basis_2d))]) 
        eta*X[i,i] + (1-eta)/2 == sum([alpha[j]*X_basis_2d[j][i,i] for j in range(len(X_basis_2d))]) 

    ]

prob = cp.Problem(cp.Maximize(eta),
                  constraints)

prob.solve(solver=cp.MOSEK, verbose=True)



MOSEK warning 500: The license expires in 5 days.
MOSEK warning 500: The license expires in 5 days.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 420             
  Cones                  : 0               
  Scalar variables       : 601             
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 420             
  Cones                  : 0               
  Scalar variables       : 601             
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  

1.0163399606367327

In [13]:
X

array([[1.        +0.00000000e+00j, 0.78213454+0.00000000e+00j,
        0.20940433+4.59498222e-02j, 0.55177645-1.81578872e-01j,
        0.20940433-4.59498222e-02j, 0.11267887+0.00000000e+00j,
        0.03203802-1.26137299e-01j, 0.55177645+1.81578872e-01j,
        0.03203802+1.26137299e-01j, 0.69910983+0.00000000e+00j],
       [0.78213454+0.00000000e+00j, 0.78213454+0.00000000e+00j,
        0.40789867-1.38777878e-17j, 0.49619719-1.38777878e-17j,
        0.20940433+4.59498222e-02j, 0.20940433+4.59498222e-02j,
        0.07154625+9.38545908e-02j, 0.55177645-1.81578872e-01j,
        0.1109781 -2.21350785e-01j, 0.55177645-1.81578872e-01j],
       [0.20940433-4.59498222e-02j, 0.40789867+1.38777878e-17j,
        0.40789867+0.00000000e+00j, 0.17123617-1.45244806e-01j,
        0.1115878 +2.44858332e-02j, 0.20940433+4.59498222e-02j,
        0.07096926+1.55728634e-02j, 0.28350825-8.69906335e-02j,
        0.1109781 -2.21350785e-01j, 0.1109781 -2.21350785e-01j],
       [0.55177645+1.81578872e-01j, 0